In [1]:
# On terminal: conda activate python38

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import ipynb.fs.defs.functions as fct
import pickle

import warnings
warnings.filterwarnings("error")

In [3]:
# Load IDs
with open('uniqueIDs.pkl', 'rb') as f:
    uniqueIDs = pickle.load(f)

# Fit mod8 for each participant

In [4]:
# Model settings

# Functions
value_fct = fct.rescorla_wagner_shrinking_alpha
dec_fct = fct.my_softmax_shrinking_press_bias

# Store everything
mod_info = {}
mod_info['name'] = 'model8'
mod_info['value_fct'] = value_fct.__name__
mod_info['dec_fct'] = dec_fct.__name__
mod_info['param_names'] = ['v0', 'alpha_t', 'beta', 'pi_t']
#print(mod['value_fct'].__name__)

# save
all_users_folder = 'data/all_users/mod8/'
file_name = all_users_folder+'mod_parameters.pkl'
with open(file_name, 'wb') as f:
    pickle.dump(mod_info, f)

In [5]:
run_ = True

# Folder
all_users_folder = 'data/all_users/mod8/'
    
if run_:
    
    # Parameter range for initial guess 
    # order of mod['param_names']:
    # ['v0', 'alpha_t', 'beta', 'pi_t']
    param_lower_bound = [-5, 0, 0, -10]
    param_upper_bound = [5, 1, 15, 10]

    # Fit
    all_users = {}
    p_hit_per_trial = pd.DataFrame([])
    ev_per_trial = pd.DataFrame([])
    trialsNos = pd.DataFrame([])
    hits = pd.DataFrame([])
    fbs = pd.DataFrame([])
    PEs = pd.DataFrame([])
    shrink_PIs = pd.DataFrame([])
    shrink_alphas = pd.DataFrame([])
    
    for n_part,ID in enumerate(uniqueIDs): 
        
        print(ID)

        # Get data
        user_folder = 'data/user_' + ID + '/'
        df2_cf = pd.read_pickle(user_folder + 'df2_cf.pkl')
        isHit_all_cues, fbs_all_cues, trialNo_all_cues = fct.extract_hits_fbs(df2_cf)

        # Create a new Model object
        mod = fct.Model(mod_name = mod_info['name'],
                     value_fct = value_fct, 
                     dec_fct = dec_fct, 
                     param_names = mod_info['param_names'])

        # Input data to model
        mod.set_data(ID, fbs_all_cues, isHit_all_cues, trialNo_all_cues)

        # Fit model
        mod.fit(param_lower_bound, param_upper_bound, n_iterations=5)

        # Nested dictionnary user data
        all_users[n_part] = {}
        all_users[n_part]['ID']=mod.ID
        all_users[n_part]['nLL']=mod.nLL
        all_users[n_part]['Ntrials']=mod.Ntrials
        all_users[n_part]['Nparams']=len(mod.param_names)
        for i in range(0,len(mod.param_names)):
            all_users[n_part][mod.param_names[i]]=mod.param_values[i]
        
        # Concatenated model predictions: p hit
        tmp = pd.DataFrame(mod.p_hit).transpose()
        tmp.columns = tmp.columns+1
        tmp = tmp.reset_index().rename(columns={'index': 'Cue'})
        tmp.insert(0,'ID',ID)
        p_hit_per_trial = pd.concat([tmp, p_hit_per_trial], axis=0)
        
        # Concatenated model predictions: EVs
        tmp = pd.DataFrame(mod.v).transpose()
        tmp.columns = tmp.columns+1
        tmp = tmp.reset_index().rename(columns={'index': 'Cue'})
        tmp.insert(0,'ID',ID)
        ev_per_trial = pd.concat([tmp, ev_per_trial], axis=0)
        
        # Concatenated trial numbers (of each cue)
        tmp = pd.DataFrame(mod.trialNo_all_cues).transpose()
        tmp.columns = tmp.columns+1
        tmp = tmp.reset_index().rename(columns={'index': 'Cue'})
        tmp.insert(0,'ID',ID)
        trialsNos = pd.concat([tmp, trialsNos], axis=0)
        
        # Concatenated fbs (of each cue)
        tmp = pd.DataFrame(mod.fbs_all_cues).transpose()
        tmp.columns = tmp.columns+1
        tmp = tmp.reset_index().rename(columns={'index': 'Cue'})
        tmp.insert(0,'ID',ID)
        fbs = pd.concat([tmp, fbs], axis=0)
        
        # Concatenated hits (of each cue)
        tmp = pd.DataFrame(mod.isHit_all_cues).transpose()
        tmp.columns = tmp.columns+1
        tmp = tmp.reset_index().rename(columns={'index': 'Cue'})
        tmp.insert(0,'ID',ID)
        hits = pd.concat([tmp, hits], axis=0)
        
        # Concatenated model predictions: PEs
        tmp = pd.DataFrame(mod.PEs).transpose()
        tmp.columns = tmp.columns+1
        tmp = tmp.reset_index().rename(columns={'index': 'Cue'})
        tmp.insert(0,'ID',ID)
        PEs = pd.concat([tmp, PEs], axis=0)
        
        # Concatenated model predictions: shrinking pi
        tmp = pd.DataFrame(mod.shrink_pi).transpose()
        tmp.columns = tmp.columns+1
        tmp = tmp.reset_index().rename(columns={'index': 'Cue'})
        tmp.insert(0,'ID',ID)
        shrink_PIs = pd.concat([tmp, shrink_PIs], axis=0)
        
        # Concatenated model predictions: shrinking alpha
        tmp = pd.DataFrame(mod.shrink_alpha).transpose()
        tmp.columns = tmp.columns+1
        tmp = tmp.reset_index().rename(columns={'index': 'Cue'})
        tmp.insert(0,'ID',ID)
        shrink_alphas = pd.concat([tmp, shrink_alphas], axis=0)
    
    
    # Save mod LLs and parameter values
    mod_fit = pd.DataFrame(all_users).transpose()
    mod_fit.to_pickle(all_users_folder+'mod_param_fits.pkl')
    
    # Save mod predictions
    p_hit_per_trial = p_hit_per_trial.sort_values(by='ID').reset_index(drop=True)
    p_hit_per_trial.to_pickle(all_users_folder+'mod_p_hit_per_trial.pkl')
    ev_per_trial = ev_per_trial.sort_values(by='ID').reset_index(drop=True)
    ev_per_trial.to_pickle(all_users_folder+'mod_ev_per_trial.pkl')
    PEs = PEs.sort_values(by='ID').reset_index(drop=True)
    PEs.to_pickle(all_users_folder+'PEs.pkl')
    shrink_PIs = shrink_PIs.sort_values(by='ID').reset_index(drop=True)
    shrink_PIs.to_pickle(all_users_folder+'shrink_PIs.pkl')
    shrink_alphas = shrink_alphas.sort_values(by='ID').reset_index(drop=True)
    shrink_alphas.to_pickle(all_users_folder+'shrink_alphas.pkl')
    
    # Save behaviour
    trialsNos = trialsNos.sort_values(by='ID').reset_index(drop=True)
    trialsNos.to_pickle(all_users_folder+'trialsNos.pkl')
    fbs = fbs.sort_values(by='ID').reset_index(drop=True)
    fbs.to_pickle(all_users_folder+'fbs.pkl')
    hits = hits.sort_values(by='ID').reset_index(drop=True)
    hits.to_pickle(all_users_folder+'hits.pkl')
    

001
003
006
007
008
009
010
012
014
023
028
032
033
034
035
036
038
044
045
049
050
054
055
057
058
060
072
073
075
076
077
078
080
084
085
086
088
090
093
096
100
101
103
104
106
107
108
109
112
114
115
118
119
120
121
122
130
132
133
136
137
138
142
143
145
147
149
151
155
158
159
165
170
172
173
175
176
177
178
181
185
186
187
189
199
204
207
208
211
213
217
221
229
230
236
237
239
240
244
251
252
258
259
263
269
271
280
282
283
284
286
287
291
292
293
301
302
303
306
313
314
315
319
321
322
325
326
327
329
335
336
337
339
341
345
349
351
360
361
362
368
375
376
381
384
390
391
393
395
397
400
405
406
412
413
414
421
422
423
427
437
438
440
446
450
453
462
469
470
471


In [6]:
mod.v

{'HR': array([-1.87752734,  4.10658892,  4.84192411, -2.48713687,  2.97098764,
        -2.77738192,  2.77040199,  0.11044335,  0.71709248,  0.90337345,
         3.2796819 ,  4.25060755,  1.73996947,  3.19657261,  2.23234778,
        -0.77234372, -0.85800126, -0.85800126, -0.85800126, -0.89584623,
        -0.89584623, -0.89584623, -0.89584623, -0.89584623, -0.89584623,
        -0.89584623, -0.89584623, -0.89584623]),
 'LR': array([-1.87752734, -1.1965376 , -1.1965376 , -1.1965376 , -1.1965376 ,
        -1.1965376 , -1.1965376 , -1.1965376 , -1.1965376 , -1.1965376 ,
        -1.1965376 , -1.1965376 , -1.1965376 , -1.1965376 , -1.1965376 ,
        -0.24954617,  1.97253663,  3.20657016,  2.32444007,  1.80534485,
         0.92573321,  0.94610867,  0.06052182,  0.1783506 ,  0.1783506 ,
         0.13216685,  0.01147811,  0.0505817 ]),
 'HP': array([-1.87752734, -4.5209531 , -4.5209531 , -4.5209531 , -4.5209531 ,
        -4.5209531 , -4.5209531 , -4.5209531 , -4.5209531 , -4.5209531 ,
        

In [7]:
mod.PEs

{'HR': array([ 6.87752734,  0.89341108, -9.84192411,  7.48713687, -7.97098764,
         7.77738192, -3.77040199,  0.88955665,  0.28290752,  4.09662655,
         1.7203181 , -5.25060755,  3.26003053, -2.19657261, -7.23234778,
        -0.22765628,         nan,         nan, -0.14199874,         nan,
                nan,         nan,         nan,         nan,         nan,
                nan,         nan,         nan]),
 'LR': array([ 0.87752734,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,  2.1965376 ,
         5.24954617,  3.02746337, -2.20657016, -1.32444007, -2.80534485,
         0.07426679, -5.94610867,  0.93947818,         nan, -1.1783506 ,
        -5.13216685,  4.98852189,         nan]),
 'HP': array([-3.12247266,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
        

In [14]:
shrink_PIs

,ID,Cue,1,2,3,4,5,6,7,8,...,19,20,21,22,23,24,25,26,27,28
0,001,LP,-3.022058,-2.887744,-2.820587,-2.551960,-2.484803,-2.417646,-2.149019,-1.947548,...,-2.619117,-1.880391,-1.544607,-1.477450,-1.410294,-1.343137,-1.074509,-0.470098,-0.402941,-0.067157
1,001,HR,-3.626469,-3.424999,-3.156371,-2.954901,-2.081862,-2.014705,-1.880391,-1.813235,...,-2.551960,-2.417646,-2.216176,-2.149019,-2.081862,-1.275980,-1.141666,-0.738725,-0.335784,-0.268627
2,001,HP,-3.693626,-3.492156,-3.357842,-3.290685,-3.223528,-2.753430,-2.350489,-2.216176,...,-2.350489,-2.014705,-1.813235,-1.746078,-1.007353,-0.940196,-0.873039,-0.805882,-0.671568,-0.604412
3,001,LR,-3.559312,-3.089215,-2.686274,-2.619117,-2.283333,-1.544607,-1.410294,-1.343137,...,-2.686274,-2.283333,-1.947548,-1.678921,-1.611764,-1.208823,-0.537255,-0.201471,-0.134314,-0.000000
4,003,HR,0.479896,0.433752,0.424523,0.415294,0.378379,0.369151,0.350693,0.332236,...,0.350693,0.304549,0.276863,0.267634,0.258405,0.212262,0.184575,0.129203,0.110745,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,470,HR,1.769743,1.461961,1.346543,1.154180,1.115707,1.000289,0.923344,0.730981,...,1.692797,1.423489,1.038762,0.884871,0.769453,0.500145,0.423199,0.346254,0.153891,0.000000
676,471,LP,2.592363,2.544356,2.400336,2.208309,2.160302,2.112296,2.016282,1.968276,...,1.776249,1.536215,1.440202,1.344188,1.152161,0.960134,0.624087,0.288040,0.192027,0.096013
677,471,HP,2.496350,2.448343,2.304323,2.256316,1.824255,1.200168,0.912128,0.624087,...,1.728242,1.584222,1.296181,1.248175,1.200168,1.056148,0.864121,0.720101,0.528074,0.432060
678,471,LR,2.064289,1.920269,1.584222,1.296181,1.248175,1.152161,1.104155,1.056148,...,2.400336,1.920269,1.680235,0.912128,0.768108,0.384054,0.240034,0.144020,0.048007,0.000000
